BraTS Challenge Demo-Notebook

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
from data_loading import BraTS_Dataset
from dataset_utils import split_cube, slice_cube, segment_entire_3d_cube
from dataset_utils import plot_batch, plot_minicube_pred_label, plot_cube_pred_label, plot_loss, animate_cube
from data_loading import get_train_test_iters
torch.manual_seed(42)
from Architectures.unet_3d import UNet3D
from Architectures.unet_3d_context import UNet3D_Mini
from Architectures.unet_2d import UNet2D
from train import train_model
from custom_losses import get_loss, DiceLoss, FocalTverskyLoss

Data Loading

In [ ]:
batch_size = 1
train_iter, test_iter = get_train_test_iters(os.path.join('..', 'Task01_BrainTumour', 'cropped'), batch_size=batch_size, shuffle=True, num_workers=0)

In [ ]:
batch = split_cube(train_iter.next(), add_context=False)
batch['image'].shape

Create the model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

Training Loop

In [ ]:
# hyperparameters
train_3d = True
add_context = False
compute_test_loss = True
epochs = 2
batches_per_epoch = 1
learning_rate = 0.1

# init model
if train_3d:
    if add_context:
        model = UNet3D_Mini(num_modalities=4, num_classes=4).to(device)
    else:
        model = UNet3D(num_modalities=4, num_classes=4).to(device)
else:
    model = UNet2D().to(device)

# init optimizer and loss_fn
optim = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = DiceLoss()

print(f'training {model.__class__.__name__}:')
train_losses, test_losses = train_model(model, optim, loss_fn, epochs, device, train_3d, add_context,
                                        train_iter, test_iter, compute_test_loss, batches_per_epoch)

Plot losses

In [ ]:
print(f'Loss curve using {loss_fn}:')
plot_loss(train_losses, test_losses)

Load inference model

In [ ]:
weights_path = os.path.join('..','Weights')
weights_filename = os.listdir(weights_path)[len(os.listdir(weights_path))-1]
if train_3d:
    if add_context:
        inference_model = UNet3D_Mini(num_modalities=4, num_classes=4).to(device)
    else:
        inference_model = UNet3D(num_modalities=4, num_classes=4).to(device)
else:
    inference_model = UNet2D().to(device)
#inference_model.load_state_dict(torch.load(os.path.join(weights_path, weights_filename)))
inference_model.eval();

Predict

In [ ]:
batch = train_iter.next()
%matplotlib notebook

ani = animate_cube(inference_model, batch, add_context, device, train_3d)
plt.show()